In [ ]:
%matplotlib inline 
# plots graphs within the notebook
%config InlineBackend.figure_format='svg' # not sure what this does, may be default images to svg format
from __future__ import division
from IPython.display import Image

from IPython.core.display import HTML
def header(text):
    raw_html = '<h4>' + str(text) + '</h4>'
    return raw_html

def box(text):
    raw_html = '<div style="border:1px dotted black;padding:2em;">'+str(text)+'</div>'
    return HTML(raw_html)

def nobox(text):
    raw_html = '<p>'+str(text)+'</p>'
    return HTML(raw_html)

def addContent(raw_html):
    global htmlContent
    htmlContent += raw_html

### Library for thermodynamic properties

Defines thermodynamic properties of air and water at 1 atm. Air properties are tabled between $-150\text{$^\circ$C}$ and $400\text{$^\circ$C}$, water between $274\text{K}$ and $373\text{K}$, Argon between $100\text{K}$ and $700\text{K}$ and Krypton between $150\text{K}$ and $750\text{K}$
<p class='alert alert-danger'>
<b>Input must be in Kelvin</b>
</p>
Use the scipy functions <FONT FACE="courier" style="color:blue">C2K </FONT> and <FONT FACE="courier" style="color:blue">F2K </FONT> to convert temperatures to Kelvin. Thermodynamic properties are linearly interpolated from the two nearest temperature states.

In [ ]:
%%file thermodynamics.py
""" Object name: Fluid"""
import sys  
sys.path.insert(0, 'Tables/') 
import numpy as np
import scipy
import scipy.optimize
from scipy.constants import convert_temperature
def C2K(T):
    return convert_temperature(T,'Celsius','Kelvin')
def C2F(T):
    return convert_temperature(T,'Celsius','Fahrenheit')
def F2K(T):
    return convert_temperature(T,'Fahrenheit','Kelvin')
def F2C(T):
    return convert_temperature(T,'Fahrenheit','Celsius')
def K2F(T):
    return convert_temperature(T,'Kelvin','Fahrenheit')
def K2C(T):
    return convert_temperature(T,'Kelvin','Celsius')
import scipy.constants as sc

def interpolate_table(target,index,xquantity,yquantity):
    return yquantity[index] + \
                (yquantity[index+1]-yquantity[index])* \
                (target-xquantity[index])/(xquantity[index+1]-xquantity[index])
        
class Fluid(object):
    """ How to:
        from NewLibraries import thermodynamics as thermo
        
        fluid_of_interest = thermo.Fluid(material,T) material can be air, water, argon and krypton (see below for ranges)
        and the temperature of the fluid T is in Kelvin.
        Outputs:
        The new object computes thermodynamic properties of air between -150 C and 400 C, 
        water between 274K and 373K, argon between 100 and 700K and
        krypton between 150 and 700 K under 1 atm. Argon, krypton and water were obtained 
        through http://webbook.nist.gov/chemistry/fluid/
        More fluids to be added in the future
        fluid_of_interest.beta thermal expansion coefficient
        fluid_of_interest.rho density
        fluid_of_interest.Cp specific heat
        fluid_of_interest.mu dynamic viscosity
        fluid_of_interest.k thermal conductivity
        fluid_of_interest.nu kinematic viscosity
        fluid_of_interest.alpha thermal diffusivity
        fluid_of_interest.Pr
        
        
        """
    def __init__(self,name,T,unit = "K",P = 101325.01):
        dirTables = 'https://raw.githubusercontent.com/yvesdubief/UVM-ME144-Heat-Transfer/master/Libraries/Tables/'
        self.name = name
        if unit == "C":
            T = C2K(T)
        elif unit == "F":
            T = F2K(T)
        self.T = T
        self.P = P
        if P != 101325.01:
            print("All available tables are for P=1ATM, reverting to P=101325.01Pa")
            self.P = 101325.01
        if self.name == 'water':
            if T < 274 or T > 373:
                print("Temperature is out of bounds for liquid water")
                return
            url = dirTables+'Water1atm.csv'
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.genfromtxt(url, delimiter=',', skip_header = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'argon':
            if T < 100 or T > 700:
                print("Temperature is out of bounds for argon")
                return
            url = dirTables + 'Argon1atm.csv'
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.loadtxt(url, delimiter=',', skiprows = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'krypton':
            if T < 150 or T > 740:
                print("Temperature is out of bounds for krypton")
                return
            url = dirTables + 'Krypton1atm.csv'
            Ttab,ptab,rhotab,Cptab,mutab,ktab = \
            np.loadtxt(url, delimiter=',', skiprows = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Cptab *= 1e3
            nutab = mutab/rhotab 
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            dTtab = Ttab[1] - Ttab[0]
            # compute beta from -rho(d rho/dT)
            betatab = -(1./rhotab)*np.gradient(rhotab)/dTtab
            i = int((T-Ttab[0])/dTtab)
            if (i == Ntab - 1):
                i == Ntab - 2
        elif self.name == 'air':
            if T < C2K(-150.) or T > C2K(400.):
                print("Temperature is out of bounds of the table for air")
                return
            url = dirTables + 'Air1atm.csv'
#             url = 'https://raw.githubusercontent.com/yvesdubief/UVM-ME144-Heat-Transfer/master/Libraries/Tables/Air1atm.csv'
#             url = 'Tables/Air1atm.csv'
            Ttab,rhotab,Cptab,ktab,nutab,betatab,Prtab = \
            np.genfromtxt(url, delimiter=',', skip_header = 1, unpack=True, dtype=float)
            Ntab = len(Ttab)
            Ttab = C2K(Ttab)
            Cptab *= 1e3
            nutab *= 1e-6
            mutab = rhotab*nutab
            alphatab = ktab/(rhotab*Cptab)
            Prtab = nutab/alphatab
            i = 0
            while (Ttab[i] < T) and (i<Ntab):
                i += 1
            i -=1
            if (i == Ntab - 1):
                i = Ntab - 2
            
        else:
            print("warning, no table available for", self.name)
            return
        
        self.rho = interpolate_table(T,i,Ttab,rhotab)
        self.Cp = interpolate_table(T,i,Ttab,Cptab)
        self.mu = interpolate_table(T,i,Ttab,mutab)
        self.k = interpolate_table(T,i,Ttab,ktab)
        self.nu = interpolate_table(T,i,Ttab,nutab)
        self.alpha = interpolate_table(T,i,Ttab,alphatab)
        self.Pr = interpolate_table(T,i,Ttab,Prtab)
        if (self.name == 'air'):
            self.beta = 1./T
        else:
            self.beta = interpolate_table(T,i,Ttab,betatab)
        

In [ ]:
import thermodynamics as thermo
import numpy as np

fluid = thermo.Fluid('water',10.,"C")
print(fluid.name)

print(fluid.k)

### Library of thermal resistances

In [ ]:
%%file HT_thermal_resistance.py
"""Object name: Resistance
   Function name: serial_sum(R,nori,nend), performs serial sum of a resistance object list from nori to nend
   Function name: parallel_sum(R,nori,nend), performs parallel sum of a resistance object list from nori to nend
   """
### definition of thermal resistance ###
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')


from IPython.display import display,Image, Latex
import numpy as np
import math
import scipy.constants as sc

import sympy as sym
#from sympy import *

class Resistance(object):
    """ Defines thermal resistances for conduction, convection and radiation heat transfer. 
        First define the object attached with class with the name used in the thermal circuit
        and the units, which can only be 'W', 'W/m' or 'W/m^2'
        Second use self.conduction, self.convection or self.radiation to calculate your 
        resistance. Each mode requires different arguments:
        
        from Libraries import HT_thermal_resistance as res
        R = []
        R.append(res.Resistance("$label$", "units")) where units = 'W', 'W/m' or 'W/m^2'
        
        then
        
        For conduction, there are 3 options:
        
        - R.cond_plane(k, L, A = 1.0) for planar conduction: k is the thermal conductivity,
                L is the thickness of the wall, and A is the optional surface area (=1 by default)
        - R.cond_cylinder(k , ra, rb, L = 1.0, angle = 2.*math.pi) for conduction in a 
                cylindrical shell between the radii ra (internal) and rb (external). L is the length
                of the shell (optional, default = 1) and angle is angular dimension of shell, also 
                optional and set to a full revolution by default (2 pi)
        - R.cond_sphere(k, ra, rb, scale = 1.0) for conductuion within a spherical shell bounded by radii ra and rb
            ra < rb. The optional parameter scale allows to calculate the thermal resistance for a fraction
            of a spherical shell. For instance a cornea is about 1/3 of spherical shell, so scale = 1./3.
        
        Convection:
        - R.convection(h, A = 1.0), where h is the convection coefficient (W/m^2K) and A is 
        the surface area (optional, default is unit surface aera 1 m^2)
        
        Radiation:
        - R.radiation(eps, T_s, T_sur, A = 1.0), where eps is the permissivity of the material, T_s
        the surface temperature, T_sur the far away surface temperature, A the surface area (optional, 
        by default A is the unit surface area 1 m^2).
        
        Contact:
        
        - R.contact(R,A,R_name= "R_{t}",A_name = "A",T_a_name = "T_a",Tb_name = "T_b"), where R is the contact resistance, typically obtained from a table
        A is the surface area
        The minimum number of arguments are:
        R.contact(R,A)
        
        R.display_equation(index) displays the heat flux/rate equations for a given resistance. index is the number of 
        your resistance (you specify)
        
        Outputs:
        - R[i].R the resistance of element i, R[i].h the convection or radiation coefficient.
        
        Functions include
        R_tot = res.serial_sum(R,first_resistance,last_resistance) sums serial resistance
        R_tot = res.parallel_sum(R,first_resistance,last_resistance) sums parallel resistance
        
        
        
        """
    def __init__(self,name,units):
        self.name = name
        self.units = units
    def cond_plane(self, k, L, A = 1.0):
        self.mode = "conduction"
        self.geometry = "planar"
        self.k = k
        if k <= 0.:
            print("problem with the definition of thermal conductivity")
        self.L = L
        self.A = A
        self.R = self.L / (self.k * self.A)
    def cond_cylinder(self, k , ra, rb, L = 1.0, angle = 2.*math.pi):
        self.mode = "conduction"
        self.geometry = "cylindrical"
        self.k = k
        if k <= 0.:
            print("problem with the definition of thermal conductivity")
        self.ra = ra
        self.rb = rb
        if ra*rb <= 0.:
            print("problem with the definition of radii")
        self.L = L
        self.angle = angle
        self.R = np.log(rb/ra)/(angle*L*k)
    def cond_sphere(self, k, ra, rb, scale = 1.0):
        self.mode = "conduction"
        self.geometry = "spherical"
        self.k = k
        if k <= 0.:
            print("problem with the definition of thermal conductivity")
        self.ra = ra
        self.rb = rb
        if ra*rb <= 0.:
            print("problem with the definition of radii")
        self.R = (1./r_a-1./r_b)/(scale*4.*math.pi*k)   
    def convection(self, h, A = 1.0):
        self.mode = 'convection'
        self.geometry = "whatever"
        self.R = 1./(h*A)
        self.A = A
        self.h = h
    def radiation(self,eps,T_s,T_sur, A = 1.0):
        self.R = 1./(eps*sc.sigma*(T_s+T_sur)*(T_s**2+T_sur**2)*A)
        self.mode = 'radiation'
        self.geometry = "whatever"
        self.A = A
        self.h = eps*sc.sigma*(T_s+T_sur)*(T_s**2+T_sur**2)
    def contact(self, R, A=1.0):
        self.R = R/A
        self.geometry = 'whatever'
        self.mode = 'contact'
        
    
        
### summation of thermal resistance (R is a vector) ###
def serial_sum(R,nori,nend):
    sum = 0.
    for i in range(nori,nend+1):
        sum += R[i].R
    return sum

def parallel_sum(R,nori,nend):
    sum = 0.
    for i in range(nori,nend+1):
        sum += 1./R[i].R
    return 1./sum




In [ ]:
import HT_thermal_resistance as res
from HT_thermal_resistance import Resistance,serial_sum,parallel_sum

Rth = []
Rth.append(res.Resistance('$R_{cond,1}$','W/m^2'))
k = 0.1 #W.m^-1.K^-1
L = 0.05 #m
A =0.8*0.5


#Rth[0].conduction('plane',k,ra,rb,A,k_name,ra_name,rb_name,A_name,Ta_name,Tb_name)
Rth[0].cond_plane(k=k,L=L,A=A)
Rth.append(Resistance('$R_{conv,1}$','W/m^2'))
h = 10. #W.m^-2.K^-1

Rth[1].convection(h,A)
for i in range(2):
    print(i,Rth[i].name,Rth[i].mode,Rth[i].R)

#print(parallel_sum(Rth,0,1))
    
#R_total = serial_sum(Rth[:].R)
#print(R_total)

<h3> Library for extended surfaces</h3>

In [ ]:
%%file HT_conduction_extended_surfaces.py
"""Object: ExtSurfaces"""
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')


from IPython.display import display,Image, Latex
import numpy as np
import math
import scipy.constants as sc

import sympy as sym
#from sympy import *

class ExtSurfaces(object):
    """ Defines temperature distribution, heat rate for constant cross sectional area fins.
        from Libraries import HT_conduction_extended_surfaces as condext
        
        fin = condext.ExtSurfaces(T_b,T_infty,T_L,k,h,P,Ac,L)
            calculates fin.m, fin.M which are constants used in flux calculation. Also provides
            fin.theta_b,.theta_L,.T_b,.T_infty,.T_L,.h,.k,.h,.P,.Ac,.L,.Af(fin exposed surface area)
        fin.heat_rate(bc) calculate the heat rate for bc="convection", "adiabatic", "isothermal", "infinite"
            The ouptuts are fin.q_f, fin.effectiveness, fin.resistance, fin.efficiency
        fin.temperature(bc,x) calculates the temperature as a function of bc and the location x
            The output is fin.theta_over_theta_b
        fin.equations(T_b_name,T_infty_name,T_L_name,k_name,h_name,P_name,Ac_name,L_name) writes all the equations for you
            you need to run fin.heat_rate first.
    """
    def __init__(self,T_b,T_infty,T_L,k,h,P,Ac,L):
        self.T_b = T_b
        self.T_infty = T_infty
        self.T_L = T_L
        theta_b = T_b-T_infty
        theta_L = T_L-T_infty
        self.theta_b = T_b-T_infty
        self.theta_L = T_L-T_infty
        self.k = k
        self.h = h
        self.P = P
        self.Ac = Ac
        self.L = L
        self.Af = self.P*self.L
        m = np.sqrt(self.h*self.P/(self.k*self.Ac))
        self.m = m
        M = np.sqrt(self.h*self.P*self.k*self.Ac)*self.theta_b
        self.M = M
    def heat_rate(self,bc):
        self.bc = bc
        it_works = True
        if self.bc == "convection":
            self.q_f = self.M*(np.sinh(self.m*self.L) + (self.h/(self.m*self.k))*np.cosh(self.m*self.L))/\
                    (np.cosh(self.m*self.L) + (self.h/(self.m*self.k))*np.sinh(self.m*self.L))
        elif self.bc == "adiabatic":
            self.q_f = self.M*np.tanh(self.m*self.L)
        elif self.bc == "isothermal":
            self.q_f = self.M*np.cosh(self.m*self.L - self.theta_L/self.theta_b)/np.sinh(self.m*self.L)
        elif self.bc == 'infinite':
            self.q_f = self.M
        else:
            print("boundary condition is not properly defined")
            it_works = False
        if it_works:
            self.effectiveness = self.q_f/(self.h*self.Ac*self.theta_b)
            self.Resistance = self.theta_b/self.q_f
            self.efficiency = self.q_f/(self.h*self.Af*self.theta_b)
        
            
    def temperature(self,bc,x):
        self.bc = bc
        if self.bc == "convection":
            self.theta_over_theta_b = (np.cosh(self.m*(self.L-x)) + (self.h/(self.m*self.k))*np.sinh(self.m*(self.L-x)))/\
                    (np.cosh(self.m*self.L) + (self.h/(self.m*self.k))*np.sinh(self.m*self.L))
        elif self.bc == "adiabatic":
            self.theta_over_theta_b = np.cosh(self.m*(self.L-x))/np.cosh(self.m*self.L)
        elif self.bc == "isothermal":
            self.theta_over_theta_b = ((self.theta_L/self.theta_b)*np.sinh(self.m*self.L)+np.sinh(self.m*self.L - x))\
                                        /np.sinh(self.m*self.L)
        elif self.bc == 'infinite':
            self.theta_over_theta_b = np.exp(-self.m*x)
        else:
            print("boundary condition is not properly defined")
        self.T_x = self.T_infty + self.theta_over_theta_b*self.theta_b
        
    
            
            
        

In [ ]:
import  HT_conduction_extended_surfaces as extsurf
h = 100.
D = 0.005
P = np.pi*D
k = 398.
Ac = np.pi*(D**2)/4
T_b = 100.
T_infty = 25.
T_L = 0.
L = 0.04
fin = extsurf.ExtSurfaces(T_b,T_infty,T_L,k,h,P,Ac,L)
fin.heat_rate('isothermal')
print(fin.q_f)
fin.temperature('infinite',L/2.)
print(fin.theta_over_theta_b,fin.T_x)

### Library of Nu correlations for external flows

In [ ]:
%%file HT_external_convection.py
""" Object name 1: FlatPlate
    Object name 2: CircularCylinder
    Object name 3: NoncircularCylinder
    Object name 4: BankofTubes
"""

from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')


from IPython.display import display,Image, Latex
import numpy as np
import math
import scipy.constants as sc
import sys
try:
    import thermodynamics as thermo
except ModuleNotFoundError:
    from Libraries import thermodynamics as thermo
import sympy as sym
#from sympy import *

class FlatPlate(object):
    """ Definition of boundary layer thickness, friction coefficient, Nusselt number (both local and average)
        as a function of the regime.
        import HT_external_convection.py as extconv
        
        bl =extconv.FlatPlate(regime,thermal_bc,U_infty,nu,alpha,L,xi=0.0,Re_xc=5e5)
            where regime = 'laminar' or 'turbulent' or 'mixed', 
            thermal_bc = 'isothermal', 'heat flux', 'unheated starting length',
            U_infty is the free stream velocity,
            nu the fluid viscosity,
            alpha the fluid thermal diffusivity,
            L length of the plate
            xi unheated started length (only applies of using unheated starting length)
            Re_xc critical Reynolds number for transition laminar to turbulence
            
        output: bl.Re_L Reynolds at the trailing edge of the plate (x=L)
        
        bl.local(x) calculates the local Re (bl.Re_x), Cf (bl.Cf_x), Nu (bl.Nu_x) and velocity
                    thermal boundary layer thicknesses (bl.delta_x and bl.delta_Tx) at x based on thermal_bc
        
        bl.average(x) calculates the average Cf (bl.C_fave), Nu (bl.Nu_ave) over a length x from the leading edge
        
        """
    def __init__(self,regime,thermal_bc,U_infty,nu,alpha,L,xi=0.0,Re_xc=5e5):
        self.regime = regime
        self.thermal_bc = thermal_bc
        self.U_infty = U_infty
        self.nu = nu
        self.alpha = alpha
        self.Pr = self.nu/self.alpha
        self.L = L
        self.xi = xi
        self.Re_xc = Re_xc
        self.Re_L = self.L*self.U_infty/self.nu
        self.x_c = self.Re_xc*self.nu/self.U_infty
        if (self.regime != "laminar") and (self.regime != "turbulent") and (self.regime != "mixed"):
            print("Warning: regime is not properly defined")
        if self.thermal_bc != "isothermal" and self.thermal_bc != "heat flux" and self.thermal_bc != "unheated starting length":
            print("Warning: thermal boundary condition is not properly defined")
        if self.Re_L > self.Re_xc and self.regime == "laminar":
            print("Warning: The end plate Reynolds number is larger than the critical Reynolds number, consider 'mixed' regime instead")
    def local(self,x):
        self.x = x
        self.Re_x = self.U_infty*self.x/self.nu
        if x == 0.:
            self.delta_x = 0.
            self.delta_Tx = 0.
            self.Cf_x = 0.
            self.Nu_x = 0.
        else:
            if self.regime == "laminar":
                self.delta_x = 5.0*self.x/np.sqrt(self.Re_x)
                self.Cf_x = 0.664*np.power(self.Re_x,-1./2.)
                if self.thermal_bc == "isothermal":
                    self.Nu_x = 0.332*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
                elif self.thermal_bc == "heat flux":
                    self.Nu_x = 0.453*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
                elif self.thermal_bc == "unheated starting length":
                    self.Re_xi = self.xi*self.U_infty/self.nu 
                    self.Nu_x = 0.332*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)/\
                            np.power(1.-np.power(self.xi/self.x,3./4.),1./3.)
            elif self.regime == "turbulent":
                self.delta_x = 0.37*self.x*np.power(self.Re_x,-1./5.)
                self.Cf_x = 0.0592*np.power(self.Re_x,-1./5.)
                if self.thermal_bc == "isothermal":
                    self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
                elif self.thermal_bc == "heat flux":
                    self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
                elif self.thermal_bc == "unheated starting length":
                    self.Re_xi = self.xi*self.U_infty/self.nu
                    self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)/\
                            np.power(1.-np.power(self.xi/self.x,9./10.),1./9.)
            elif self.regime == "mixed":
                if self.x < self.x_c:
                    self.delta_x = 5.0*self.x/np.sqrt(self.Re_x)
                    self.Cf_x = 0.664*np.power(self.Re_x,-1./2.)
                    if self.thermal_bc == "isothermal":
                        self.Nu_x = 0.332*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
                    elif self.thermal_bc == "heat flux":
                        self.Nu_x = 0.453*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
                    elif self.thermal_bc == "unheated starting length":
                        self.Re_xi = self.xi*self.U_infty/self.nu 
                        self.Nu_x = 0.332*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)/\
                            np.power(1.-np.power(self.xi/self.x,3./4.),1./3.)
                else:
                    self.delta_x = 0.37*self.x*np.power(self.Re_x,-1./5.)
                    self.Cf_x = 0.0592*np.power(self.Re_x,-1./5.)
                    if self.thermal_bc == "isothermal":
                        self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
                    elif self.thermal_bc == "heat flux":
                        self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
                    elif self.thermal_bc == "unheated starting length":
                        self.Re_xi = self.xi*self.U_infty/self.nu
                        self.Nu_x = 0.0296*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)/\
                            np.power(1.-np.power(self.xi/self.x,9./10.),1./9.)
                        
            self.delta_Tx = self.delta_x*np.power(self.Pr,-1./3.)
    def average(self,x):
        self.x = x
        self.Re_x = self.U_infty*self.x/self.nu
        if x == 0.:
            print("The length cannot be zero")
        if self.regime == "laminar":
            self.Cf_ave = 1.328*np.power(self.Re_x,-1./2.)
            if self.thermal_bc == "isothermal" or self.thermal_bc == "heat flux":
                self.Nu_ave = 0.664*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)
            elif self.thermal_bc == "unheated starting length":
                p = 2.
                self.Re_xi = self.xi*self.U_infty/self.nu
                self.Nu_ave = 0.664*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)*\
                              x/(x-self.xi)*np.power(1.-np.power(self.xi/x,(p+1.)/(p+2.)),p/(p+1.))
        elif self.regime == "turbulent":
            self.Cf_ave = 0.074*np.power(self.Re_x,-1./5.)
            if self.thermal_bc == "isothermal" or self.thermal_bc == "heat flux":
                self.Nu_ave = 0.037*np.power(self.Re_x,4./5.)*np.power(self.Pr,1./3.)
            elif self.thermal_bc == "unheated starting length":
                p = 8.
                self.Re_xi = self.xi*self.U_infty/self.nu
                self.Nu_ave = 0.664*np.power(self.Re_x,1./2.)*np.power(self.Pr,1./3.)*\
                              x/(x-self.xi)*np.power(1.-np.power(self.xi/x,(p+1.)/(p+2.)),p/(p+1.))
        elif self.regime == "mixed":
            A = 0.037*np.power(self.Re_xc,4./5.)-0.664*np.power(self.Re_xc,1./2.)
            
            self.C_fave = 0.074*np.power(self.Re_x,-1./5.) - 2.*A/self.Re_x
            self.Nu_ave = (0.037*np.power(self.Re_x,4./5.) - A)*np.power(self.Pr,1./3.)
                
class CircularCylinder(object):
    """ Nusselt correlations for cylinders
    import HT_external_convection.py as extconv
        
    bluff_body =extconv.CircularCylinder(correlation,Re,Pr,Pr_s = 0.0) 
        where Re, Pr, and Pr_s are the Reynolds number, Prandtl number of the flow and surface Prandtl numbers, respectively. If using Hilpert of Churchill Bernstein correlations,
        Re and Pr must be defined at film temperature, Pr_s can be set to anything since it is not used. 
        If using Zukauskas, Re and Pr are defined at temperature at infinity.
        correlation may be 'Hilpert', 'Churchill-Bernstein', 'Zukauskas'
        Example:
        bluff_body = extconv.CircularCylinder('Hilpert',Re,Pr)
        bluff_body = extconv.CircularCylinder('Churchill-Bernstein',Re,Pr)
        bluff_body = extconv.CircularCylinder('Zukauskas',Re,Pr,Pr_s = xx)
        
        Output: bluff_body.Nu average Nusselt number also bluff_body.correlation, bluff_body.Re, bluff_body.Pr, bluff_body.Pr_s
    
    bluff_body.correlation('Name of the correlation')
    Name of the correlation may be 'Hilpert', 'Churchill-Bernstein', 'Zukauskas'
    
    
    """
    def __init__(self,correlation,Re,Pr,Pr_s = 0.0):
        self.correlation = correlation
        self.Re = Re
        self.Pr = Pr
        self.Pr_s = Pr_s
        if correlation == "Zukauskas" and Pr_s == 0.0:
            print("Warning: Zukauskas correlation requires Pr_s")
        if self.correlation == "Hilpert":
            if self.Re < 0.4:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = 0.
            elif  self.Re < 4.:
                C = 0.989
                m = 0.33
            elif  self.Re < 40:
                C = 0.911
                m = 0.385
            elif self.Re < 4000:
                C = 0.683
                m = 0.466
            elif self.Re < 40000.:
                C = 0.193
                m = 0.618
            elif self.Re <= 400000.:
                C = 0.027
                m = 0.805
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = 0.
            if self.Re >= 0.4 and self.Re <= 400000.:
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
        elif self.correlation == "Churchill-Bernstein":
            if (self.Re*self.Pr < 0.2):
                print("Warning: Product RePr lower than acceptable limit for Churchill Bernstein Correlation")
                self.Nu = 0.
    
            else:
                self.Nu = 0.3+(0.62*self.Re**(0.5)*self.Pr**(1./3.)) \
                  /(1.+(0.4/self.Pr)**(2./3.))**(1./4.) \
                *(1.+(self.Re/282000.)**(5./8.))**(4./5.)
        elif self.correlation == "Zukauskas":
            if (self.Pr <= 10):
                n = 0.37
            else:
                n = 0.36
            if (self.Re < 1.) and (self.Re > 1.e6):
                print("Warning Reynolds number out of bounds for the Zukauskas Correlation")
                self.Nu = 0.
            else:
                if (self.Re < 40.):
                    C = 0.75
                    m = 0.4
                elif (self.Re < 1000.):
                    C = 0.51
                    m = 0.5
                elif (self.Re < 2.e5):
                    C = 0.26
                    m = 0.6
                else:
                    C = 0.076
                    m = 0.7
                self.Nu = C*self.Re**m*self.Pr**n*(self.Pr/self.Pr_s)**(1./4.)

class NonCircularCylinder(object):
    """ Nusselt correlations for  cylinders with non circular cross-sections.
    import HT_external_convection.py as extconv
        
    bluff_body =extconv.NonCircularCylinder(geometry,Re,Pr) where 
    geometry = "angled square" square with stagnation point on one of its edges
               "square" square with stagnation point at the center of one of its faces
               "angled hexagon" hexagon with stagnation point on one of its edges
               "hexagon" hexagon with stagnation point at the center of one of its faces
               "thin plate" thin plate perpendicular to the flow
    Re: Reynolds number at film temperature
    Pr: Prandtl number at film temperature
    
    Output: bluff_body.Nu, bluff_body.Nu_front, bluff_body.Nu_back, the last two are for thin plate only
    also bluff_body.geometry, bluff_body.Re, bluff_body.Pr
    
    """
    def __init__(self,geometry,Re,Pr):
        self.geometry = geometry
        self.Re = Re
        self.Pr = Pr
        if self.geometry == "angled square":
            self.Nu_front = np.inf
            self.Nu_back = np.inf
            if self.Re < 6000:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = np.inf
            elif  self.Re <= 60000.:
                C = 0.304
                m = 0.59
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = np.inf
        elif self.geometry == "square":
            self.Nu_front = np.inf
            self.Nu_back = np.inf
            if self.Re < 5000:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = np.inf
            elif  self.Re <= 60000.:
                C = 0.158
                m = 0.66
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = np.inf
        elif self.geometry == "angled hexagon":
            self.Nu_front = np.inf
            self.Nu_back = np.inf
            if self.Re < 4500:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = np.inf
            elif  self.Re <= 90700.:
                C = 0.150
                m = 0.638
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = np.inf
        elif self.geometry == "hexagon":
            self.Nu_front = np.inf
            self.Nu_back = np.inf
            if self.Re < 5200:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu = np.inf
            elif  self.Re <= 20400.:
                C = 0.164
                m = 0.638
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            elif  self.Re <= 105000.:
                C = 0.039
                m = 0.78
                self.Nu = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation")
                self.Nu = np.inf
        elif self.geometry == "thin plate":
            self.Nu = np.inf
            if self.Re < 10000:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu_front = np.inf
            elif  self.Re <= 50000.:
                C = 0.667
                m = 0.5
                self.Nu_front = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation for Nu_front")
                self.Nu_back = np.inf
            if self.Re < 7000:
                print("Warning, Reynolds number too low for Hilpert Correlation")
                self.Nu_back = np.inf
            elif  self.Re <= 80000.:
                C = 0.191
                m = 0.667
                self.Nu_back = C * self.Re**m * self.Pr**(1./3.)
            else :
                print("Warning Reynolds number is too high for the Hilpert Correlation for Nu_front")
                self.Nu_back = np.inf
                
class BankofTubes(object):
    """ Nusselt correlations for flow across banks of tubes
    import HT_external_convection.py as extconv
    bank = extconv.BankofTubes('aligned','air',T_i,T_s,T_o,"C",V_i,D,S_L,S_T,N_L=1)
    **Input:** 
    arrangement = "aligned" tubes are aligned in row and column
                  "staggered" tubes are staggered from one row to the next
    fluidspecie = 'air', 'water' or any other specie available in library thermodynamics
    T_i = inlet temperature
    T_s = tube surface temperature
    T_o = outlet temperature, provide a guess < T_s if unknown
    V_i: Inlet velocity
    S_L: tube center to tube center separation  between two consecutive rows (perpendicular to the flow)
    S_T: tube center to tube center separation  between two consecutive rows (aligned with the flow)
    N_L: number of rows aligned with flow if unknown give your best guess (default =1)
    
    Output:
    bank.T_m: arithmetic mean of T_i and T_o
    bank.Nu: average Nusselt number
    bank.Delta_T_lm: Log mean temperature difference
    bank.rho_i: inlet fluid density
    bank.mu_m: fluid viscosity at T_m
    bank.k_m: fluid thermal conductivity at T_m
    bank.Cp_m: Specific heat at T_m
    bank.Pr_m: Prandtl number at T_m
    bank.Pr_s: Prandtl number at T_s
    bank.Vmax: Max fluid velocity based on arrangement
    bank.Re: Reynolds number of the system
    
    bank.Nu: average Nusselt number
    bank.hbar: average heat transfer coefficient
    
    Functions:        
    bank.heat_rate(N_T,N_L,L=1) creates bank.q, the heat rate per tube length is L is omitted  
                or total heat rate if L is provided, based on the average convection coefficient
    bank.temperature_outlet_tube_banks(N_T,N_L) overwite bank.T_o (useful if T_o is unkown and you provided a 
        guess. Rerun the object with new T_o to adjust Nu)
    bank.pressure_drop(N_L,f,chi) creates bank.Delta_p the pressure drop across the bank. f and chi must
        be extrapolated from graphs (see book or notebook)
    

    """
    def __init__(self,arrangement,fluidspecie,T_i,T_s,T_o,unit,V_i,D,S_L,S_T,N_L=1):
        self.arrangement = arrangement
        self.T_i = T_i
        self.T_s = T_s
        self.V_i = V_i
        if (T_o == T_s):
            print("T_o and T_s cannot be equal, setting T_o to (T_i+T_s)/2 (think of it as your first guess)")
            T_o = (T_i+T_s)/2.
        self.T_o = T_o
        T_m = (T_i + T_o)/2.
        self.T_m = T_m
        self.Delta_T_lm = ((T_s-T_i)-(T_s-T_o))/np.log((T_s-T_i)/(T_s-T_o))
        self.fluid = fluidspecie
        fluid_i = thermo.Fluid(fluidspecie,T_i,unit)
        fluid_m = thermo.Fluid(fluidspecie,T_m,unit)
        fluid_s = thermo.Fluid(fluidspecie,T_s,unit)
        self.rho_i = fluid_i.rho
        self.mu_m = fluid_m.mu
        self.k_m = fluid_m.k
        self.Cp_m = fluid_m.Cp
        self.Pr_m = fluid_m.nu/fluid_m.alpha
        self.Pr_s = fluid_s.nu/fluid_s.alpha
        self.S_L = S_L
        self.S_T = S_T
        self.N_L = N_L
        self.D = D
        if self.arrangement == 'aligned':
            self.Vmax = self.S_T*self.V_i/(self.S_T-D)
        elif self.arrangement == 'staggered':
            self.S_D = np.sqrt(self.S_L**2+(self.S_T/2.)**2)
            self.Vmax = self.S_T*V_i/(2.*(self.S_D-D))
        Re = self.rho_i*self.Vmax*self.D/self.mu_m
        self.Re = Re
        self.Nu = np.inf
        Corr_aligned = np.array([0.70,0.80,0.86,0.90,0.92,0.94,0.95,0.96,0.96,0.97,0.97,0.97,0.98,0.99,0.99,0.99,0.99,0.99,0.99])
        Corr_staggered = np.array([0.64,0.76,0.84,0.89,0.92,0.94,0.95,0.96,0.96,0.97,0.97,0.97,0.98,0.99,0.99,0.99,0.99,0.99,0.99])
        if (N_L < 20):
            if arrangement == 'aligned':
                Corr = Corr_aligned[N_L-1]
            elif arrangement == 'staggered':
                Corr = Corr_staggered[N_L-1]
        else:
            Corr = 1.
        if (Re < 10.):
            print('Warning: Re is out of bounds')
        if (Re >= 10.) and (Re <= 100.):
            if arrangement == 'aligned':
                C = 0.8
                m = 0.4
            elif arrangement == 'staggered':
                C = 0.9
                m = 0.4
            self.Nu = Corr*C*self.Re**m*self.Pr_m**(0.36)*(self.Pr_m/self.Pr_s)**(1./4.)
        elif (Re > 100.) and (Re <= 1000.):
            C = 0.51
            m = 0.
            self.Nu = Corr*C*self.Re**m*self.Pr_m**(0.36)*(self.Pr_m/self.Pr_s)**(1./4.)
        elif (Re > 1000.) and (Re <= 2.e5):
            if arrangement == 'aligned':
                if (S_T/S_L > 0.7):
                    C = 0.27
                    m = 0.63
                else:
                    print('Warning: inefficient, S_T/S_L<0.7')
                
            elif arrangement == 'staggered':
                if (S_T/S_L < 2):
                    C = 0.35*(S_T/S_L)**(1./5.)
                    m = 0.6
                else:
                    C = 0.40
                    m = 0.6
            self.Nu = Corr*C*self.Re**m*self.Pr_m**(0.36)*(self.Pr_m/self.Pr_s)**(1./4.)
        elif (Re > 2e5) and (Re <= 2.e6):
            if arrangement == 'aligned':
                C = 0.021
                m = 0.84
            elif arrangement == 'staggered':
                C = 0.022
                m = 0.84
            self.Nu = Corr*C*self.Re**m*self.Pr_m**(0.36)*(self.Pr_m/self.Pr_s)**(1./4.)
        else:
            print('Warning: Re is out of bounds')

        self.hbar = self.Nu*self.k_m/self.D
        self.N_L_for_given_To = -np.log((self.T_s-self.T_o)/(self.T_s-self.T_i))/ \
            (np.pi*self.D*self.hbar)*(self.rho_i*self.V_i*self.S_T*self.Cp_m)
        if (self.N_L < 20) and (self.N_L_for_given_To >= 20):
            print("WARNING input N_L < 20 but N_L computed for input T_o >=20. \ Rerun your BankofTubes object with the appropriate N_L to calculate the correct Nu")
        if (self.N_L < 20) and (self.N_L_for_given_To >= 20):
            print("WARNING input N_L >= 20 but N_L computed for input T_o <20. \ Rerun your BankofTubes object with the appropriate N_L to calculate the correct Nu")

    def heat_rate(self,N_T,N_L,L=1):
        N = N_T*N_L
        if N_L > 20 and self.N_L < 20:
            print("WARNING: you chose N_L > 20 but your initial guess was < 20. \ Rerun your BankofTubes object with the appropriate N_L to calculate the correct Nu")
        elif N_L < 20 and self.N_L >= 20:
            print("WARNING: you chose N_L < 20 but your initial guess was > 20. \ Rerun your BankofTubes object with the appropriate N_L to calculate the correct Nu")
        self.q=N*self.hbar*np.pi*self.D*self.Delta_T_lm*L

            
    def temperature_outlet_tube_banks(self,N_T,N_L):
        if N_L >= 20 and self.N_L < 20:
            print("WARNING: you chose N_L > 20 but your initial guess was < 20. \ Rerun your BankofTubes object with the appropriate N_L to calculate the correct Nu")
        elif N_L < 20 and self.N_L >= 20:
            print("WARNING: you chose N_L < 20 but your initial guess was > 20. \ Rerun your BankofTubes object with the appropriate N_L to calculate the correct Nu")
        N = N_T*N_L
        self.T_o = self.T_s-(self.T_s-self.T_i)* \
            np.exp(-np.pi*self.D*N*self.hbar/(self.rho_i*self.V_i*N_T*self.S_T*self.Cp_m))
    def pressure_drop(self,N_L,f,chi):
        self.Delta_p = N_L*chi*(self.rho_i*self.Vmax**2/2)*f







In [ ]:


import HT_external_convection as extconv

square = extconv.NonCircularCylinder("thin plate",30000.,0.7)
print(square.Nu_back)

In [ ]:



D = 10.e-3
S_T = 15.e-3
S_L = S_T
N_L = 14
N_T = 14
N = 196
T_i = 25.
V_i = 5.
T_s = 100.
T_o = T_s - 10.
L = 1.
import thermodynamics as thermo
bank = extconv.BankofTubes('aligned','air',T_i,T_s,T_o,"C",V_i,D,S_L,S_T,N_L)

print(bank.Re)
print(bank.Nu)
print(bank.hbar)
bank.heat_rate(N_T,N_L,L)
print(bank.q)
bank.temperature_outlet_tube_banks(N_T,N_L)
print(bank.T_o)
T_o = bank.T_o
print("New iteration")
bank = extconv.BankofTubes('aligned','air',T_i,T_s,T_o,"C",V_i,D,S_L,S_T,N_L)

print(bank.Re)
print(bank.Nu)
print(bank.hbar)
bank.heat_rate(N_T,N_L,L)
print(bank.q)
bank.temperature_outlet_tube_banks(N_T,N_L)
print(bank.T_o)
bank.pressure_drop(N_L,0.32,1.)
print(bank.Delta_p)
# h = bank.Nu*air.k/D
# T_o = extconv.temperature_outlet_tube_banks(T_s,T_i,D,N,N_T,h,air.rho,V_i,S_T,air.Cp)
# DT_lm = extconv.Delta_T_lm(T_s,T_i,T_o)
# bank.heat_rate(h,D,T_s,T_i,T_o)
# print(bank.q)
# print(T_o)

In [ ]:



D = 10.e-3
S_T = 15.e-3
S_L = S_T
N_L = 14
N_T = 14
N = 196
T_i = 25.
V_i = 5.
T_s = 100.
T_o = 75.
L = 1.
import thermodynamics as thermo
bank = extconv.BankofTubes('aligned','air',T_i,T_s,T_o,"C",V_i,D,S_L,S_T,N_L)

print(bank.Re)
print(bank.Nu)
print(bank.hbar)
print(bank.N_L_for_given_To)
N_L = int(bank.N_L_for_given_To) + 1
bank.heat_rate(N_T,N_L,L)
print(bank.q)
bank.temperature_outlet_tube_banks(N_T,N_L)
print(bank.T_o)
# T_o = bank.T_o
# print("New iteration")
# bank = extconv.BankofTubes('aligned','air',T_i,T_m,T_s,T_o,"C",V_i,D,S_L,S_T,N_L)

# print(bank.Re)
# print(bank.Nu)
# print(bank.hbar)
# bank.heat_rate(N_T,N_L,L)
# print(bank.q)
# bank.temperature_outlet_tube_banks(N_T,N_L)
# print(bank.T_o)
# bank.pressure_drop(N_L,0.32,1.)
# print(bank.Delta_p)
# # h = bank.Nu*air.k/D
# # T_o = extconv.temperature_outlet_tube_banks(T_s,T_i,D,N,N_T,h,air.rho,V_i,S_T,air.Cp)
# # DT_lm = extconv.Delta_T_lm(T_s,T_i,T_o)
# # bank.heat_rate(h,D,T_s,T_i,T_o)
# # print(bank.q)
# # print(T_o)

### Library of Nu correlations and functions for internal flow in pipes

In [ ]:
%%file HT_internal_convection.py
""" 
Object name: PipeFlow
"""
import numpy as np
import scipy
import scipy.optimize

class PipeFlow(object):
    """ Determination of Nu, pressure drop, mean temperature for internal convection
        import HT_internal_convection.py as intconv
        
        pipe =intconv.PipeFlow(D, Re=0.0, Um = 0.0, mdot = 0.0, nu = 0.0, rho = 0.0)
        where 
        D is the only required input and one of the following combination (Re, nu) or (Um, nu) or (mdot, rho, nu)
        Hence the minimum calls for PipeFlow are
        pipe =intconv.PipeFlow(D, Re= Re_m, nu = nu_m) outputs pipe.Um
        pipe =intconv.PipeFlow(D, Re= Re_m, nu = nu_m, rho = rho_m) outputs pipe.Um (bulk velocity) 
                and pipe.mdot (mass flow)
        pipe =intconv.PipeFlow(D, Um = 0.0, nu = 0.0) outputs pipe.Re
        pipe =intconv.PipeFlow(D, Um = Um, nu = nu_m, rho = rho_m) outputs pipe.Re, pipe.mdot
        pipe =intconv.PipeFlow(D, mdot = 0.0, nu = 0.0, rho = 0.0) outputs pipe.Re, pipe.Um
        
        pipe.f_laminar(Re) outputs the friction factor for laminar flow pipe.f
        pipe.f_turbulent(Re,eps = 0.0, nu = 0.0) outputs the friction factor for turbulent flow pipe.f
        
        The following correlations output pipe.Nu
        pipe.laminar_isothermal for isothermal wall boundary condition
        pipe.laminar_isoflux for isoflux wall boundary condition
        pipe.Dittus_Boelter(mode, Pr, Re = 0.) for turbulent flow where mode is either "heating" or "cooling"
            The Re is optional if omitted, the Reynolds number calculated in the object PipeFlow will be used
        pipe.Sieder_Tate(Pr,mu,mu_s, Re = 0.0) mu and mu_s are the mean and wall dynamics viscosities
            The Re is optional if omitted, the Reynolds number calculated in the object PipeFlow will be used
        pipe.Gnielinski( Pr, f,Re = 0.0): where f is the friction factor
            The Re is optional if omitted, the Reynolds number calculated in the object PipeFlow will be used
        
        """
    def __init__(self,D, Re=0.0, Um = 0.0 , mdot = 0.0, nu = 0.0, rho = 0.0, L = 1.0 ):
        self.D = D
        self.L = L
            
        if Re == 0.0:
            if Um != 0.0 and nu != 0.0:
                Re = Um*D/nu
            elif mdot != 0 and rho != 0.0 and nu != 0.0:
                Um = mdot/(rho*np.pi*D**2/4.)
                Re = Um*D/nu
            else:
                print("Warning if Re == 0, Um, D and nu or mdot, rho and nu must be specified")
                
        self.Re = Re
        if Um == 0.:
            if Re != 0. and nu != 0.:
                Um = Re*nu/D
                if mdot == 0.0 and rho != 0.0:
                    mdot = rho*Um*np.pi*D**2/4.
            elif mdot !=0.0 and rho != 0.0:
                Um = mdot/(rho*np.pi*D**2/4.)
                 
                
        self.Um = Um
        if mdot == 0.0:
            if rho != 0.0:
                mdot = rho*Um*np.pi*D**2/4.
            else:
                self.rho = 1.0
                self.mdot = rho*Um*np.pi*D**2/4.
        self.mdot = mdot
        self.nu = nu
        if Re == 0. and nu != 0.:
            Re = Um*D/nu
        self.Re = Re
        
        if rho == 0.0:
            self.rho = 1.0
            
        else:
            self.rho = rho
            
    


    def f_laminar(self, Re = 0.0):
        if Re == 0. and self.Re !=0:
            Re = self.Re
        elif Re == 0 and self.Re == 0.0:
            print("Warning Reynolds number is not defined")
        self.f = 64./Re
        self.dPdx = self.f*(self.L/self.D)*(self.rho*self.Um**2)/2.

    def f_turbulent(self,Re = 0.0, eps = 0.0):
        if Re == 0. and self.Re !=0.0:
            Re = self.Re
        elif Re == 0 and self.Re == 0.0:
            print("Warning Reynolds number is not defined")
        if eps == 0.0:
            print("Pipe wall is assumed to be hydrodynamically smooth") 
        e = eps
     
        f_0 = (0.790*np.log(Re)- 1.64)**(-2.)
        if (e > 0.):
            f_1 = 1./(-2.0*np.log10(e/3.71))**2
        else:
            f_1 = f_0
        f_guess = np.max([f_0,f_1])
        #f_guess = 0.04
        def f_tmp(x):
            y = (-2*np.log10((2.51/(Re*np.sqrt(x))) + (e/(3.71))) - 1.0/np.sqrt(x))
            return y
        y = scipy.optimize.fsolve(f_tmp, f_guess)
        self.f = y[0]
        self.dPdx = self.f*(self.L/self.D)*(self.rho*self.Um**2)/2.
        
    def laminar_isothermal(self):
        self.Nu = 3.66

    def laminar_isoflux(self):
        self.Nu = 4.36

    def Dittus_Boelter(self,mode,Pr,Re = 0.0):
        if Re == 0. and self.Re !=0:
            Re = self.Re
        else:
            print("Warning Reynolds number is not defined")
        if (mode == 'heating'):
            n = 0.4
        elif (mode == 'cooling'):
            n = 0.3
        else:
            print("Warning you have to specify mode='heating' or 'cooling'")
        self.Nu = 0.023*Re**(4./5.)*Pr**n

    def Sieder_Tate(self,Pr,mu,mu_s, Re = 0.0):
        if Re == 0. and self.Re !=0:
            Re = self.Re
        else:
            print("Warning Reynolds number is not defined")
        self.Nu = 0.027*Re**(4/5)*Pr**(1/3)*(mu/mu_s)**0.14

    def Gnielinski(self, Pr, f,Re = 0.0):
        if Re == 0. and self.Re !=0:
            Re = self.Re
        else:
            print("Warning Reynolds number is not defined")
        self.Nu = (f/8.)*(Re-1000.)*Pr/(1+12.7*(f/8.)**0.5*(Pr**(2./3.)-1.))

    def Skupinski(self,Pr, Re = 0.0):
        if Re == 0. and self.Re !=0:
            Re = self.Re
        else:
            print("Warning Reynolds number is not defined")
        self.Nu = 4.82+0.0185*(Re*Pr)**0.827

    def Seban(self,Pr, Re = 0.0):
        if Re == 0. and self.Re !=0:
            Re = self.Re
        else:
            print("Warning Reynolds number is not defined")
        self.Nu = 5.0+0.025*(Re*Pr)**0.8

def log_mean_temperature(T_s,T_o,T_i):
    if (T_s < min(T_o,T_i)):
        DT_o = T_o-T_s
        DT_i = T_i-T_s
    elif (T_s > max(T_o,T_i)):
        DT_o = T_s-T_o
        DT_i = T_s-T_i
    return (DT_o-DT_i)/np.log(DT_o/DT_i)


def T_mx_Ts_constant(T_s,T_mi,P,mdot,Cp,hbar,x):
    return T_s-(T_s-T_mi)*np.exp(-P*x*hbar/(mdot*Cp))

def T_mo_T_infty(T_infty,T_mi,mdot,Cp,R_tot):
    return T_infty-(Tinfty-T_mi)*np.exp(-1/(mdot*Cp*Rtot))

def L_given_other_params(T_infty,T_mo,T_mi,mdot,Cp,Rptot):
    return -mdot*Cp*Rptot*np.log((T_infty -T_mo)/(T_infty - T_mi))



In [ ]:
import HT_internal_convection as intconv

pipe = intconv.PipeFlow(D=0.01, L= 1., rho=1000., nu=1e-6, Um=2)

pipe.f_turbulent()
print(pipe.f,pipe.dPdx, pipe.Re)

### Library for natural convection around cylinders

In [5]:
%%file HT_natural_convection.py
""" 
Object name: 
    - HorizontalCylinder
    - FlatPlate
    - VerticalEnclosure
Functions: Gr(g,beta,DT,D,nu) gives the Grashoff number based on:
            gravity g, thermal expansion coefficient beta, Temperature difference DT, 
            length scale D, viscosity nu
           Ra(g,beta,DT,D,nu,alpha) gives the Rayleigh number where alpha is the thermal conductivity.
"""
import numpy as np
import scipy
import scipy.optimize

class HorizontalCylinder(object):
    """ Natural convection about a horizontal cylinder
        from NewLibraries import HT_natural_convection as natconv
        cyl = natconv.HorizontalCylinder(correlation, Ra, Pr = 0.0)
        where correlation is "Morgan" or "Churchill-Chu"
        cyl = natconv.HorizontalCylinder("Morgan", Ra)
        cyl = natconv.HorizontalCylinder("Churchill-Chu", Ra, Pr = xx)
    """

    def __init__(self,correlation="Morgan", Ra=0.0, Pr = 0.0):
        self.correlation = correlation
        self.Ra = Ra
        if correlation == "Morgan":
            if (Ra <= 1e-2):
                C=0.675
                n=0.058
            elif (Ra <= 1e2):
                C=1.02
                n=0.148
            elif (Ra <= 1e4):
                C=0.85
                n=0.188
            elif (Ra <= 1e7):
                C=0.480
                n=0.250
            elif (Ra <= 1e12):
                C=0.125
                n=0.333
            self.Nu = C*Ra**n
        elif correlation == "Churchill-Chu":
            if Pr == 0.:
                print("Warning you must specify Pr for Churchill and Chu correlation")
            else:
                self.Nu = (0.60+(0.387*Ra**(1./6.))/(1.+(0.559/Pr)**(9./16.))**(8./27.))**2
        else:
            print("Warning wrong correlation name")

class VerticalEnclosure(object):
    """ Natural convection about a horizontal cylinder
        from NewLibraries import HT_natural_convection as natconv
        cyl = natconv.HorizontalCylinder(correlation, Ra, Pr = 0.0)
        where correlation is "Morgan" or "Churchill-Chu"
        cyl = natconv.HorizontalCylinder("Morgan", Ra)
        cyl = natconv.HorizontalCylinder("Churchill-Chu", Ra, Pr = xx)
    """

    def __init__(self,Ra,Pr,H,L):
        self.Ra = Ra
        self.Pr = Pr
        self.H = H
        self.L = L
        if correlation == "Morgan":
            if (H/L) < 2.:
                if Ra*Pr/(0.2+Pr)> 1.e3:
                    self.Nu = 0.18*(Pr/(0.2+Pr)*Ra)**0.29
                else:
                    print('Ra is too low for this correlation')
                    self.Nu = np.inf
            elif H/L < 10:
                if Ra < 1e10:
                    self.Nu = 0.22*(Pr/(0.2+Pr)*Ra)**0.28*(H/L)**(-0.25)
                else:
                    print('Ra is too high for this correlation')
                    self.Nu = np.inf
            elif Ra < 1e4:
                print('Ra is too low for this correlation')
                self.Nu = np.inf
            elif Ra < 1e7:
                if Pr > 0.6 and Pr < 2e4:
                    print('ok')
                    self.Nu =0.42*Ra**0.25*Pr**0.012*(H/L)**(-0.3)
                else :
                    print('Pr is out of bounds for this correlation')
                    self.Nu = np.inf
            elif Ra < 1e9:
                if Pr > 0.6 and Pr < 20.:
                    self.Nu =0.46*Ra**(1./3.)
                else :
                    print('Pr is out of bounds for this correlation')
                    self.Nu = np.inf
            else:
                print('Ra is too high, got nothing for you')
                self.Nu = np.inf
class FlatPlate(object):
    """ Natural convection caused by a flat plate at temperature T_s
        in a fluid at given ambient temperature T_infty. 
        Inputs:
        - Ra based on the |T_s - T_infty| and L=A_s/P_s where 
          A_s and P_s are the area and the perimeter of the plate,
          respectively.
        - Pr
        - surface is either 'upper' or 'lower'
        - surfaceT is either 'cold' or 'hot'
        Output:
        - Average Nu_L
        Limits:
        from NewLibraries import HT_natural_convection as natconv
        plate = natconv.FlatPlate(Ra= Ra_L, Pr = Pr, 
                                 surface = 'upper', surfaceT = 'hot')
        plate = natconv.FlatPlate(Ra= Ra_L, Pr = Pr, 
                                 surface = 'lower', surfaceT = 'cold')
        - Ra > 1e4
        - If 1e4 <= Ra <= 1e7 then Pr>= 0.7
        - If 1e7 <= Ra <= 1e11 then all Pr
        plate = natconv.FlatPlate(Ra= Ra_L, Pr = Pr, 
                                 surface = 'lower', surfaceT = 'hot')
        plate = natconv.FlatPlate(Ra= Ra_L, Pr = Pr, 
                                 surface = 'upper', surfaceT = 'cold')
        - 1e4<= Ra <= 1e9, Pr>=0.7
       
    """

    def __init__(self,Ra,Pr,surface='none',surfaceT='none'):
        self.Ra = Ra
        self.Pr = Pr
        self.surface = surface
        self.surfaceT = surfaceT
#         print(self.surface,self.surfaceT)
        if (self.surface != 'upper') and (self.surface != 'lower'):
            print(self.surface)
            print("you must specify surface='upper' or 'lower'")
        if (self.surfaceT != 'hot') and (self.surfaceT != 'cold'):
            print("you must specify surfaceT='hot' or 'cold'")
        if ((self.surface == 'upper') and (self.surfaceT == 'hot')) or \
            ((self.surface == 'lower') and (self.surfaceT == 'cold')):
                if (self.Ra >= 1e4) and (self.Ra <= 1e7):
                    if self.Pr < 0.7:
                        print("Warning: For %s surface of %s plate, \
                              the correlation is only valid for Pr>= 0.7 " %(self.surface,self.surfaceT))
                    self.Nu = 0.54*self.Ra**(1/4)
                elif (self.Ra >= 1e7) : #and (self.Ra <= 1e11):
                    self.Nu = 0.15*self.Ra**(1/3)
                else:
                    print("Ra is too small")
                    self.Nu = 0
        if ((self.surface == 'lower') and (self.surfaceT == 'hot')) or \
            ((self.surface == 'upper') and (self.surfaceT == 'cold')):
            if (self.Ra >= 1e4) : # and (self.Ra <= 1e9):
                self.Nu = 0.15*self.Ra**(1/5)
                if self.Pr < 0.7:
                    print("Warning: the correlation is only valid for Pr>= 0.7")
            else:
                print("Ra is too small")
                self.Nu=0


def Gr(g=9.81,beta=0.0,DT=0.0,D=0.0,nu=1.0):
    return (g*beta*DT*D**3)/(nu**2)

def Ra(g=9.81,beta=0.0,DT=0.0,D=0.0,nu=1.0,alpha=1.0):
    return (g*beta*DT*D**3)/(nu*alpha)





Overwriting HT_natural_convection.py


In [ ]:
import HT_natural_convection as natconv
plate = natconv.FlatPlate(1e12,1,'upper','hot')
plate.Nu